# Image Classification with TensorFlow and Keras
This notebook demonstrates an image classification pipeline using TensorFlow and Keras. The process includes data loading, image preprocessing, augmentation, model creation, training, and visualization.

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
import os
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
import random


## Data Loading and Preprocessing
In this section, we load and preprocess the image data. We apply augmentations for better model generalization.

In [2]:
# Load images from directory
train_data_dir = '/path/to/train_data'
test_data_dir = '/path/to/test_data'

# Apply ImageDataGenerator for real-time data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)


## Visualize Sample Images
Let's visualize a few images from the training dataset to understand the data better.

In [3]:
# Visualize some images from the training dataset
sample_images, sample_labels = next(train_generator)

fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for i in range(5):
    axes[i].imshow(sample_images[i])
    axes[i].axis('off')
plt.show()

## Model Architecture
Next, we define the deep learning model architecture.

In [4]:
# Define the model architecture
model = Sequential()

# Input layer to avoid warnings
model.add(Input(shape=(64, 64, 3)))

# Add layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))  # Assuming 10 classes

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


## Training the Model
We will now train the model using the training data and validate using the validation data.

In [5]:
# Set early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


## Evaluate the Model
Let's evaluate the model's performance on the validation data.

In [6]:
# Evaluate the model on validation data
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')


## Visualize Training History
We will plot the training and validation accuracy and loss to visualize the model's performance over epochs.

In [7]:
# Plot training & validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


## Save the Model
We will now save the trained model for later use.

In [8]:
# Save the model
model.save('image_classification_model.h5')

# Optionally save the model architecture to JSON
model_json = model.to_json()
with open('image_classification_model.json', 'w') as json_file:
    json_file.write(model_json)


## Conclusion
In this notebook, we created and trained an image classification model using TensorFlow and Keras. The model was evaluated and visualized, and we saved the final trained model.